<a target="_blank" href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/autosegment_CT_HeartSubStruct_SMIT.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# CT_HeartSubstruct_SMIT Demo Notebook

User supplies input DICOM files, copy `*.dcm` files to `dcmDir` folder.

In [ ]:
import os, shutil, subprocess
from glob import glob

In [ ]:
workDir = '/content'
os.chdir(workDir)

In [ ]:
!pip install pyxnat
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git"

In [ ]:
from cerr import plan_container as pc
from cerr.dcm_export import rtstruct_iod

In [ ]:
def pycerr_dcm2nii(dcmDir,niiOutDir,niiFileBase='scan.nii.gz',scanNum=0):
  planC = pc.loadDcmDir(dcmDir = dcmDir)
  niiFileName = os.path.join(niiOutDir,niiFileBase)
  planC.scan[scanNum].saveNii(niiFileName = niiFileName)
  return niiFileName, planC

In [ ]:
def pycerr_mask2rtstruct(planC, maskFile, labels_dict, rtstructFileName = 'RTSTRUCT.dcm', seriesDescription = 'Imported by pyCERR', scanNum = 0, structNumV = None):
  if structNumV == None:
    structNumV = range(len(labels_dict))
  planC = pc.loadNiiStructure(nii_file_name = maskFile, assocScanNum = scanNum, planC = planC, labels_dict = labels_dict)
  rtstruct_iod.create(structNumV = structNumV, filePath = rtstructFileName, planC = planC, seriesOpts = {'SeriesDescription':seriesDescription})
  return rtstructFileName, planC

In [ ]:
! git clone https://github.com/cerr/model_installer.git && cd model_installer && ./installer.sh -m 8 -d /content -p C &&

In [ ]:
dcmDir = '/content/session/input_dcm'
niiDir = '/content/session/input_nii'
maskOutDir = '/content/session/output_nii'
dcmOutDir = '/content/session/output_dcm'

In [ ]:
modelDir = os.path.join(workDir,'CT_HeartSubstruct_SMIT')
weightsPath = os.path.join(modelDir,'/trained_weights/model_final.pt')

In [ ]:
!mkdir -p {dcmDir} {niiDir} {maskOutDir} {dcmOutDir}

In [ ]:
scanNum = 0
niiFileName, planC = pycerr_dcm2nii(dcmDir,niiDir,niiFileBase='CT_scan.nii.gz',scanNum = scanNum)

In [ ]:
cmd = 'source ' + os.path.join(modelDir,'conda-pack','bin','activate') + ' && cd ' + modelDir + ' && source bash_run_SMIT_HeartSubstructures.sh ' + niiDir + ' ' + maskOutDir + ' ' +  niiFileName
subprocess.run(cmd, shell=True, executable="/bin/bash")

In [ ]:
labels_dict = {1: 'SMIT_AORTA',2:'SMIT_PULMONARY_ARTERY',3:'SMIT_PULMONARY_VEIN',4:'SMIT_SUP_VENA_CAVA',5:'SMIT_INF_VENA_CAVA'}

In [ ]:
outNiiFile = glob(os.path.join(maskOutDir,'*.nii*'))
print(outNiiFile)

In [ ]:
structName ='CT_HeartSubstruct_SMIT'
rtstructFile = os.path.join(dcmOutDir,'RTSTRUCT.dcm')

In [ ]:
rtstructFileName, planC = pycerr_mask2rtstruct(planC = planC, maskFile = outNiiFile, labels_dict = labels_dict, rtstructFileName = rtstructFile)